In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import os

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")

class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 1)
        )

    def forward(self, x, y):
        inputs = torch.cat([x, y], dim=1)
        return self.net(inputs)

def compute_derivatives(model, x, y):
    """
    Computes V and its first and second derivatives using Autograd.
    Ensures inputs are tracked for gradients.
    """
    # Force gradient tracking on inputs if not already set
    if not x.requires_grad: x.requires_grad_(True)
    if not y.requires_grad: y.requires_grad_(True)

    V = model(x, y)

    # First derivatives
    grads = torch.autograd.grad(V, [x, y], grad_outputs=torch.ones_like(V),
                                create_graph=True, retain_graph=True)
    dV_dx, dV_dy = grads[0], grads[1]

    # Second derivatives
    d2V_dx2 = torch.autograd.grad(dV_dx, x, grad_outputs=torch.ones_like(dV_dx),
                                  create_graph=True, retain_graph=True)[0]
    d2V_dy2 = torch.autograd.grad(dV_dy, y, grad_outputs=torch.ones_like(dV_dy),
                                  create_graph=True, retain_graph=True)[0]

    return V, dV_dx, dV_dy, d2V_dx2, d2V_dy2

def get_boundary_data(N_b, L, Vmax, k_vals, bc_type_right='dirichlet'):
    k1, k2, k3, k4 = k_vals

    pos = torch.rand(N_b, 1) * 2 * L - L
    const = torch.ones_like(pos)

    d_l = (-L*const, pos, Vmax * torch.sin(k1 * np.pi * (pos + L) / (2 * L)))

    target_r = Vmax * torch.sin(k3 * np.pi * (pos + L) / (2 * L)) if bc_type_right == 'dirichlet' else torch.zeros_like(pos)
    d_r = (L*const, pos, target_r)

    d_b = (pos, -L*const, Vmax * torch.sin(k4 * np.pi * (pos + L) / (2 * L)))
    d_t = (pos, L*const, Vmax * torch.sin(k2 * np.pi * (pos + L) / (2 * L)))

    return {'left': d_l, 'right': d_r, 'bottom': d_b, 'top': d_t}

def get_collocation_points(N_c, L):
    x_c = (torch.rand(N_c, 1) * 2 * L - L).to(device)
    y_c = (torch.rand(N_c, 1) * 2 * L - L).to(device)
    return x_c, y_c

def save_results_with_residual(model, history, task_name, L, Arho, epsilon):
    N_plot = 100
    x_np = np.linspace(-L, L, N_plot)
    y_np = np.linspace(-L, L, N_plot)
    X, Y = np.meshgrid(x_np, y_np)

    x_tensor = torch.tensor(X.flatten(), dtype=torch.float32).unsqueeze(1).to(device)
    y_tensor = torch.tensor(Y.flatten(), dtype=torch.float32).unsqueeze(1).to(device)
    x_tensor.requires_grad_(True)
    y_tensor.requires_grad_(True)

    V_flat, _, _, d2V_dx2, d2V_dy2 = compute_derivatives(model, x_tensor, y_tensor)
    rho_flat = Arho * x_tensor * y_tensor * torch.exp(-(x_tensor**2 + y_tensor**2))

    # Residual = | Laplacian(V) + rho/epsilon |
    res_flat = torch.abs(d2V_dx2 + d2V_dy2 + (rho_flat / epsilon))

    V_grid = V_flat.detach().cpu().numpy().reshape(N_plot, N_plot)
    Res_grid = res_flat.detach().cpu().numpy().reshape(N_plot, N_plot)

    fig = plt.figure(figsize=(18, 5))

    ax1 = fig.add_subplot(1, 3, 1)
    im1 = ax1.imshow(V_grid, extent=[-L, L, -L, L], origin='lower', cmap='seismic')
    ax1.set_title(f'{task_name}: Potential V(x,y)')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    plt.colorbar(im1, ax=ax1)

    ax2 = fig.add_subplot(1, 3, 2)
    im2 = ax2.imshow(Res_grid, extent=[-L, L, -L, L], origin='lower', cmap='coolwarm')
    ax2.set_title(f'{task_name}: Residual |$\nabla^2 V + \\rho$|')
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    plt.colorbar(im2, ax=ax2, format='%.1e')

    ax3 = fig.add_subplot(1, 3, 3)
    ax3.plot(history)
    ax3.set_yscale('log')
    ax3.set_title('Loss Convergence')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Loss (MSE)')
    ax3.grid(True, which="both", ls="-", alpha=0.5)

    plt.tight_layout()
    filename = f"pinn_data/PINN_{task_name}_Analysis.png"
    plt.savefig(filename)
    print(f"Saved analysis to {filename}")
    plt.show()
    plt.close()

def run_task(task_name, Arho, k_vals, bc_type_right, epochs=6000):
    print(f"\n--- Running {task_name} ---")

    L, epsilon, Vmax = 4.0, 1.0, 1.0
    model = PINN().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    loss_history = []

    for epoch in range(epochs):
        optimizer.zero_grad()

        # A. Boundary Loss
        bc_data = get_boundary_data(200, L, Vmax, k_vals, bc_type_right)
        loss_bc = 0.0

        to_dev = lambda d: (d[0].to(device), d[1].to(device), d[2].to(device))

        # Dirichlet Boundaries
        for side in ['left', 'top', 'bottom']:
            xb, yb, target = to_dev(bc_data[side])
            loss_bc += torch.mean((model(xb, yb) - target)**2)

        # Right Boundary (Fix applied here)
        xb, yb, target = to_dev(bc_data['right'])

        if bc_type_right == 'dirichlet':
            loss_bc += torch.mean((model(xb, yb) - target)**2)
        else:
            # FIX: Explicitly set requires_grad for Neumann derivatives
            xb.requires_grad_(True)
            yb.requires_grad_(True)
            _, dV_dx, _, _, _ = compute_derivatives(model, xb, yb)
            loss_bc += torch.mean((dV_dx - target)**2)

        # B. PDE Residual Loss
        xc, yc = get_collocation_points(2000, L)
        xc.requires_grad_(True)
        yc.requires_grad_(True)

        _, _, _, d2V_dx2, d2V_dy2 = compute_derivatives(model, xc, yc)
        rho = Arho * xc * yc * torch.exp(-(xc**2 + yc**2))
        residual = d2V_dx2 + d2V_dy2 + (rho / epsilon)
        loss_pde = torch.mean(residual**2)

        # Total
        loss = loss_bc + loss_pde
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())

        if epoch % 1000 == 0:
            print(f"Epoch {epoch}: Loss {loss.item():.5f}")

    save_results_with_residual(model, loss_history, task_name, L, Arho, epsilon)

if __name__ == "__main__":
    # Task 1: Laplace
    run_task("Task1", 0.0, (1, -1, 1, -1), 'dirichlet')

    # Task 2: Laplace with Neumann Right
    run_task("Task2_Neumann", 0.0, (1, -1, 1, -1), 'neumann')

    # Task 4a: Poisson (Arho=1) with Zero BC
    run_task("Task4a", 1.0, (0, 0, 0, 0), 'dirichlet')

    # Task 4b: Poisson (Arho=1) with Standard BC
    run_task("Task4b", 1.0, (1, -1, 1, -1), 'dirichlet')

    print("\nProcessing complete.")

Running on: cpu

--- Running Task1 ---
Epoch 0: Loss 2.24311
Epoch 1000: Loss 0.00075
Epoch 2000: Loss 0.00024
Epoch 3000: Loss 0.00043
Epoch 4000: Loss 0.00005
Epoch 5000: Loss 0.00005
Saved analysis to PINN_Task1_Analysis.png

--- Running Task2_Neumann ---
Epoch 0: Loss 1.82295
Epoch 1000: Loss 0.00703
Epoch 2000: Loss 0.00301
Epoch 3000: Loss 0.00132
Epoch 4000: Loss 0.00107
Epoch 5000: Loss 0.00163
Saved analysis to PINN_Task2_Neumann_Analysis.png

--- Running Task4a ---
Epoch 0: Loss 0.08035
Epoch 1000: Loss 0.00004
Epoch 2000: Loss 0.00001
Epoch 3000: Loss 0.00001
Epoch 4000: Loss 0.00001
Epoch 5000: Loss 0.00000
Saved analysis to PINN_Task4a_Analysis.png

--- Running Task4b ---
Epoch 0: Loss 1.93428
Epoch 1000: Loss 0.00404
Epoch 2000: Loss 0.00168
Epoch 3000: Loss 0.00177
Epoch 4000: Loss 0.00123
Epoch 5000: Loss 0.00087
Saved analysis to PINN_Task4b_Analysis.png

Processing complete.
